In [2]:
import selenium
print(selenium.__version__)

4.5.0


In [ ]:
# !pip install selenium
# !pip install webdriver_manager

In [22]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager # 크롬버전 상관없이 드라이버 설치 상관없이 
from selenium.webdriver.chrome.service import Service # 
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import time
import pandas as pd
import csv

In [10]:
#-- activate webdriver 
    # browser = webdriver.Chrome("../chromedriver.exe")
browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
url = "https://naver.com"
url_1 = "https://search.shopping.naver.com/"
#-- open the selected url with chrome
# browser.get(url)
browser.get(url_1)
browser.implicitly_wait(2)

In [11]:
# #-- click on the shopping menu(with. js path)
# # browser.find_element(By.CSS_SELECTOR,"#NM_FAVORITE > div.group_nav > ul.list_nav.type_fix > li:nth-child(5) > a").click() 
# browser.find_element(By.CSS_SELECTOR,"a.nav.shop").click()
# time.sleep(2)


In [12]:
#-- click the searchbar 
search = browser.find_element(By.CSS_SELECTOR,"#__next > div > div.header_header__24NVj > div > div > div._gnb_header_area_150KE > div > div._gnbLogo_gnb_logo_3eIAf > div > div._gnbSearch_gnb_search_3O1L2 > form > fieldset > div._gnbSearch_inner_2Zksb > div > input")
search.click()
#-- input a item to find in the search bar
search.send_keys("아이폰") # input text 
search.send_keys(Keys.ENTER) # 

In [13]:
#-- infinite scroll
def infinite():
    before_h = browser.execute_script("return window.scrollY")
    while True:
        browser.find_element(By.CSS_SELECTOR,"body").send_keys(Keys.END) # Use the END key on the keyboard to send it to the end of the scroll
        time.sleep(1) # 1sec delay  
        after_h = browser.execute_script("return window.scrollY")
        if after_h == before_h:
            break
        before_h = after_h
    
infinite()

In [ ]:
#-- Extract text
def extract_text():
    items = browser.find_elements(By.CSS_SELECTOR,".basicList_info_area__TWvzp")
    for item in items:
        name = item.find_element(By.CSS_SELECTOR,".basicList_title__VfX3c").text
        #-- Exception handling in case of ERROR in price data
        try:
            price = item.find_element(By.CSS_SELECTOR,".price_num__S2p_v").text
        except:
            price = "판매중단"
        link = item.find_element(By.CSS_SELECTOR,".basicList_title__VfX3c > a" ).get_attribute("href")
        # print(name,"\t",price,"\t",link)
extract_text()                                                              

In [24]:
f = open(r"./data.csv","w",encoding="CP949",newline="") # path  mode    encoding    line_change
csvWriter = csv.writer(f)

items = browser.find_elements(By.CSS_SELECTOR,".basicList_info_area__TWvzp")
for item in items:
    name = item.find_element(By.CSS_SELECTOR,".basicList_title__VfX3c").text
    #-- Exception handling in case of ERROR in price data
    try:
        price = item.find_element(By.CSS_SELECTOR,".price_num__S2p_v").text
    except:
        price = "판매중단"
    link = item.find_element(By.CSS_SELECTOR,".basicList_title__VfX3c > a" ).get_attribute("href")
    # print(name,"\t",price,"\t",link)
    
    csvWriter.writerow([name,price,link])
f.close()